# Book Recommendation system by clustering | collaborative based
data source:https://www.kaggle.com/datasets/ra4u12/bookrecommendation
code reference: BAPPY AHMED https://github.com/entbappy     

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os



In [63]:
# 使用 Path 處理跨作業系統的路徑問題
books = pd.read_csv(Path('data') / 'BX-Books.csv', 
                    sep=";", 
                    on_bad_lines='skip', 
                    encoding='latin-1')

/var/folders/j1/_f9dctx973g6ykl494fpd4b80000gn/T/ipykernel_33123/2251178973.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(Path('data') / 'BX-Books.csv',


In [64]:
books.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [65]:
books.shape

(271360, 8)

In [66]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [67]:
books=books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [68]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [69]:
books.rename(columns={
   "Book-Title":"title",
    "Book-Author":"author",
    "Year-Of-Publication":"year",
    "Publisher":"publisher",
    "Image-URL-L":"img_url"},inplace = True)

In [70]:
books.head(2)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [71]:
# 使用 Path 處理跨作業系統的路徑問題
users = pd.read_csv(Path('data') / 'BX-Users.csv', 
                    sep=";", 
                    on_bad_lines='skip', 
                    encoding='latin-1')#再讀取另外一個dataset

In [72]:
users.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


In [73]:
users.shape

(278858, 3)

In [74]:
# 使用 Path 處理跨作業系統的路徑問題
ratings = pd.read_csv(Path('data') / 'BX-Book-Ratings.csv', 
                    sep=";", 
                    on_bad_lines='skip', 
                    encoding='latin-1')#再讀取另外一個dataset

In [75]:
ratings.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [76]:
ratings.rename(columns={
   "User-ID":"user_id",
    "Book-Rating":"rating",
    },inplace = True)

In [77]:
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


現在有三個dataframe:books,users,ratings

In [79]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [80]:
ratings['user_id'].value_counts()#可以看到每個id的rating次數

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [81]:
ratings['user_id'].unique().shape#不太需要極端值，進行data clean
#.unique()代表找到'user_id'欄位中不重複的唯一值
#.shape 計算唯一值的數量
#所以，這行程式碼的作用是計算 ratings data set中不同的用戶 ID 總數。


(105283,)

In [82]:
x = ratings['user_id'].value_counts() > 200 #篩選出200以上的資料

In [83]:
x[x].shape #得到結果發現有899個用戶有rating超過200次
#[x] 會返回一個新的 Series，只包含 True 的值。
#所以 x[x].shape 計算的是滿足條件（出現次數 > 200）的用戶 ID 數量

(899,)

In [84]:
y= x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [85]:
ratings = ratings[ratings['user_id'].isin(y)]
#This filters the original ratings DataFrame to keep only ratings from the selected users

In [86]:
ratings.head(2)

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0


In [87]:
ratings.shape

(526356, 3)

In [88]:
ratings_with_books=ratings.merge(books, on = "ISBN")

In [89]:
ratings_with_books.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...


In [90]:
ratings_with_books.shape

(487671, 8)

In [91]:
num_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [92]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [93]:
num_rating.rename(columns={
    'rating':'num_of_rating'},inplace=True)

In [94]:
num_rating.head(2)

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1


In [95]:
final_rating = ratings_with_books.merge(num_rating,on='title')

In [96]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7


In [97]:
final_rating.shape

(487671, 9)

In [98]:
final_rating=final_rating[final_rating['num_of_rating']>=50]

In [99]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133


In [100]:
final_rating.sample(100) #隨便抽樣100個樣本

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
216548,126736,006101351X,0,The Perfect Storm : A True Story of Men Agains...,Sebastian Junger,1998,HarperTorch,http://images.amazon.com/images/P/006101351X.0...,163
51858,28204,0345342968,0,Fahrenheit 451,RAY BRADBURY,1987,Del Rey,http://images.amazon.com/images/P/0345342968.0...,119
207869,120548,0553260111,0,The Bourne Identity,Robert Ludlum,1984,Bantam Books,http://images.amazon.com/images/P/0553260111.0...,71
309505,179978,0671027360,0,Angels &amp; Demons,Dan Brown,2001,Pocket Star,http://images.amazon.com/images/P/0671027360.0...,193
183123,104665,0451176464,7,Gerald's Game,Stephen King,2001,Signet Book,http://images.amazon.com/images/P/0451176464.0...,68
...,...,...,...,...,...,...,...,...,...
83081,40943,0671673688,0,While My Pretty One Sleeps,Mary Higgins Clark,1990,Pocket,http://images.amazon.com/images/P/0671673688.0...,75
229293,130571,0316666343,8,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown",http://images.amazon.com/images/P/0316666343.0...,270
134481,76942,034541389X,0,Flesh and Blood,Jonathan Kellerman,2002,Ballantine Books,http://images.amazon.com/images/P/034541389X.0...,103
103117,55548,0060929871,0,Brave New World,Aldous Huxley,1998,Perennial,http://images.amazon.com/images/P/0060929871.0...,70


In [101]:
final_rating.shape


(61853, 9)

In [102]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)
#這一步意義在於因為可能同一個user可能會重複對同一本數rating多次，所以透過這樣的方式保留user第一次的評論
#, keeping only the first rating for each unique user-title combination

In [103]:
final_rating

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79
...,...,...,...,...,...,...,...,...,...
487505,275970,1400031354,0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031354.0...,84
487506,275970,1400031362,0,Morality for Beautiful Girls (No.1 Ladies Dete...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031362.0...,60
487579,275970,1573229725,0,Fingersmith,Sarah Waters,2002,Riverhead Books,http://images.amazon.com/images/P/1573229725.0...,59
487618,275970,1586210661,9,Me Talk Pretty One Day,David Sedaris,2001,Time Warner Audio Major,http://images.amazon.com/images/P/1586210661.0...,146


## covert to pivot-table
userA 可能與userB 的rating程度差不多，代表兩個user可以被歸類同一群

In [105]:
book_pivot = final_rating.pivot_table(columns='user_id',index='title',values = 'rating')

In [106]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [107]:
book_pivot.shape

(742, 888)

In [108]:
book_pivot.fillna(0,inplace=True)

In [109]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
from scipy.sparse import csr_matrix
# sparse matrix用來處理的大量元素中只有少數非零值的情況
# csr(Compressed Sparse Row)_martix:only non-zero elements
#常出現在推薦系統，因為推薦系統中，評分矩陣通常是稀疏的：

In [111]:
book_sparse = csr_matrix(book_pivot)

In [112]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14961 stored elements in Compressed Sparse Row format>

In [113]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')
#algorithm='brute' 直接計算所有數據點之間的距離
#algorthm還有'auto','ball_tree','kd_tree"


In [114]:
model.fit(book_sparse)
#.fit(X): 擬合數據

NearestNeighbors(algorithm='brute')

In [115]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)
#kneighbors(X): 找最近鄰居、kneighbors_graph(X): 生成鄰居、n_neighbors: 鄰居數量
#.values將 pandas Series 或 DataFrame 轉換為 NumPy 數組
#.reshape(1, -1)將數組重塑為二維數組（必要的）1 表示一行、-1 讓 NumPy 自動計算列數

In [116]:
distance

array([[ 0.        , 67.75691847, 68.05145112, 72.277244  , 75.81556568,
        76.30203143]])

In [117]:
suggestion 

array([[237, 238, 240, 241, 184, 536]])

In [118]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [119]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [120]:
book_pivot.index

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [121]:
books_name = book_pivot.index

In [122]:
import pickle
pickle.dump(model, open('/Users/zhanjieyu/Python.Jupyter/artifacts/model.pkl','wb'))
pickle.dump(books_name, open('/Users/zhanjieyu/Python.Jupyter/artifacts/books_name.pkl','wb'))
pickle.dump(final_rating, open('/Users/zhanjieyu/Python.Jupyter/artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot, open('/Users/zhanjieyu/Python.Jupyter/artifacts/book_pivot.pkl','wb'))


In [123]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)

    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [124]:
book_name='Year of Wonders'
recommend_book(book_name)
##只要輸入自己常看的書，就會推薦書

Year of Wonders
No Safe Place
A Civil Action
Pleading Guilty
The Temple of My Familiar
Sleepers
